<a href="https://colab.research.google.com/github/durgas4/PPE-detection-using-yolo8/blob/main/PPE_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


For the following code, try to use cv2.rectangle to draw rectangle around images

### **XML to text conversion and class mapping**

In [5]:
import os
import argparse
import xml.etree.ElementTree as ET

def load_class_mapping(classes_file):
    class_mapping = {}
    with open(classes_file, 'r') as file:
        for idx, line in enumerate(file):
            class_name = line.strip()
            class_mapping[class_name] = idx
    return class_mapping

def convert_voc_to_yolo(voc_path, yolo_path, class_mapping):
    if not os.path.exists(yolo_path):
        os.makedirs(yolo_path)

    for filename in os.listdir(voc_path):
        if filename.endswith(".xml"):
            # Parse the XML file
            tree = ET.parse(os.path.join(voc_path, filename))
            root = tree.getroot()

            # Get image size
            size = root.find("size")
            width = int(size.find("width").text)
            height = int(size.find("height").text)

            # Prepare YOLO format file
            yolo_filename = os.path.splitext(filename)[0] + ".txt"
            yolo_filepath = os.path.join(yolo_path, yolo_filename)

            with open(yolo_filepath, "w") as yolo_file:
                for obj in root.findall("object"):
                    class_name = obj.find("name").text
                    # Get class ID
                    class_id = class_mapping.get(class_name, -1)  # Use -1 for unknown classes

                    if class_id == -1:
                        print(f"Warning: Class '{class_name}' not found in class mapping.")
                        continue

                    bbox = obj.find("bndbox")
                    xmin = int(bbox.find("xmin").text)
                    ymin = int(bbox.find("ymin").text)
                    xmax = int(bbox.find("xmax").text)
                    ymax = int(bbox.find("ymax").text)

                    # Convert to YOLO format (normalized)
                    x_center = (xmin + xmax) / 2.0 / width
                    y_center = (ymin + ymax) / 2.0 / height
                    obj_width = (xmax - xmin) / width
                    obj_height = (ymax - ymin) / height

                    # Write to YOLO format file
                    yolo_file.write(f"{class_id} {x_center} {y_center} {obj_width} {obj_height}\n")

def main():
    parser = argparse.ArgumentParser(description="Convert Pascal VOC annotations to YOLOv8 format.")
    parser.add_argument("input_dir", type=str, help="/content/drive/MyDrive/datasets/labels (1)")
    parser.add_argument("output_dir", type=str, help="/content/drive/MyDrive/datasets/label ")
    parser.add_argument("classes_file", type=str, help="/content/drive/MyDrive/datasets/classes.txt")

    args = parser.parse_args()

    # Load class mapping from file
    class_mapping = load_class_mapping(args.classes_file)

    convert_voc_to_yolo(args.input_dir, args.output_dir, class_mapping)

if __name__ == "__main__":
    main()


usage: colab_kernel_launcher.py [-h] input_dir output_dir classes_file
colab_kernel_launcher.py: error: the following arguments are required: output_dir, classes_file


SystemExit: 2

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [6]:
import os
import xml.etree.ElementTree as ET


def load_class_mapping(mapping_file):
    class_mapping = {}
    with open(mapping_file, 'r') as file:
        for idx, line in enumerate(file):
            class_name = line.strip()
            class_mapping[class_name] = idx
    return class_mapping

import os
import xml.etree.ElementTree as ET

def load_class_mapping(mapping_file):
    class_mapping = {}
    with open(mapping_file, 'r') as file:
        for idx, line in enumerate(file):
            class_name = line.strip()
            class_mapping[class_name] = idx
    return class_mapping

def convert_voc_to_yolov8(voc_folder, yolov8_folder, class_mapping_file):
    # Create YOLOv8 folder if it doesn't exist
    if not os.path.exists(yolov8_folder):
        os.makedirs(yolov8_folder)

    # Load class mapping from file
    class_mapping = load_class_mapping(class_mapping_file)

    for filename in os.listdir(voc_folder):
        if filename.endswith(".xml"):
            xml_path = os.path.join(voc_folder, filename)
            tree = ET.parse(xml_path)
            root = tree.getroot()

            # Get image dimensions
            size = root.find("size")
            img_width = float(size.find("width").text)
            img_height = float(size.find("height").text)

            yolo_filename = os.path.splitext(filename)[0] + ".txt"
            with open(os.path.join(yolov8_folder, yolo_filename), "w") as yolo_file:
                for obj in root.findall("object"):
                    class_name = obj.find("name").text
                    if class_name in class_mapping:
                        class_id = class_mapping[class_name]  # Get the class ID from the mapping
                    else:
                        print(f"Warning: Class '{class_name}' not found in mapping file.")
                        continue

                    bbox = obj.find("bndbox")
                    x_min = float(bbox.find("xmin").text)
                    y_min = float(bbox.find("ymin").text)
                    x_max = float(bbox.find("xmax").text)
                    y_max = float(bbox.find("ymax").text)

                    # Calculate bounding box width and height
                    width = x_max - x_min
                    height = y_max - y_min

                    # Calculate normalized coordinates
                    x_center = (x_min + width / 2) / img_width
                    y_center = (y_min + height / 2) / img_height
                    width /= img_width
                    height /= img_height

                    # Write class ID and normalized coordinates to YOLO format
                    yolo_file.write(f"{class_id} {x_center} {y_center} {width} {height}\n")

            # Remove the XML file after conversion
            os.remove(xml_path)

# Example usage
voc_folder = "/content/drive/MyDrive/datasets/labels (1)"
yolov8_folder = "/content/drive/MyDrive/datasets/label"
class_mapping_file = "/content/drive/MyDrive/datasets/classes.txt"

convert_voc_to_yolov8(voc_folder, yolov8_folder, class_mapping_file)


## Train TEst split

In [8]:
import os
import shutil

def filter_annotations_and_images(source_images_dir, source_labels_dir, target_class_id, output_images_dir, output_labels_dir):
    if not os.path.exists(output_images_dir):
        os.makedirs(output_images_dir)
    if not os.path.exists(output_labels_dir):
        os.makedirs(output_labels_dir)

    label_files = [f for f in os.listdir(source_labels_dir) if f.endswith('.txt')]

    for label_file in label_files:
        label_path = os.path.join(source_labels_dir, label_file)
        image_file = label_file.replace('.txt', '.jpg')  # Adjust extension if needed
        image_path = os.path.join(source_images_dir, image_file)

        with open(label_path, 'r') as file:
            lines = file.readlines()

        filtered_lines = [line for line in lines if int(line.split()[0]) == target_class_id]

        if filtered_lines:
            if os.path.exists(image_path):
                shutil.copy(image_path, output_images_dir)

            new_label_path = os.path.join(output_labels_dir, label_file)
            with open(new_label_path, 'w') as file:
                file.writelines(filtered_lines)

def prepare_dataset(source_images_dir, source_labels_dir, target_class_id, train_images_dir, train_labels_dir, val_images_dir, val_labels_dir):
    for directory in [train_images_dir, train_labels_dir, val_images_dir, val_labels_dir]:
        if not os.path.exists(directory):
            os.makedirs(directory)

    all_label_files = [f for f in os.listdir(source_labels_dir) if f.endswith('.txt')]
    import random
    random.shuffle(all_label_files)
    split_index = int(len(all_label_files) * 0.8)
    train_label_files = all_label_files[:split_index]
    val_label_files = all_label_files[split_index:]

    for label_files, img_output_dir, lbl_output_dir in [(train_label_files, train_images_dir, train_labels_dir), (val_label_files, val_images_dir, val_labels_dir)]:
        for label_file in label_files:
            label_path = os.path.join(source_labels_dir, label_file)
            image_file = label_file.replace('.txt', '.jpg')  # Adjust extension if needed
            image_path = os.path.join(source_images_dir, image_file)

            with open(label_path, 'r') as file:
                lines = file.readlines()

            filtered_lines = [line for line in lines if int(line.split()[0]) == target_class_id]

            if filtered_lines:
                if os.path.exists(image_path):
                    shutil.copy(image_path, img_output_dir)

                new_label_path = os.path.join(lbl_output_dir, label_file)
                with open(new_label_path, 'w') as file:
                    file.writelines(filtered_lines)

# Directories and parameters
# Directories
source_images_dir = '/content/drive/MyDrive/datasets/images'
source_labels_dir = '/content/drive/MyDrive/datasets/labels'
target_class_id = 0  # ID of the class you want to detect

# Output directories for filtered data
train_images_dir = '/content/drive/MyDrive/datasets/person_Detection/train/images'
train_labels_dir = '/content/drive/MyDrive/datasets/person_Detection/train/labels'
val_images_dir = '/content/drive/MyDrive/datasets/person_Detection/val/images'
val_labels_dir = '/content/drive/MyDrive/datasets/person_Detection/val/labels'

# Prepare the dataset
prepare_dataset(source_images_dir, source_labels_dir, target_class_id, train_images_dir, train_labels_dir, val_images_dir, val_labels_dir)


In [9]:
# Load YOLOv8n-cls, train it on mnist160 for 10 epochs and predict an image with it
from ultralytics import YOLO
model = YOLO("/content/drive/MyDrive/datasets/person_Detection/yolov8n (1).pt")  # load a pretrained YOLOv8n classification model
model.train(data='/content/drive/MyDrive/datasets/person_Detection/data.yaml', epochs=10)



Ultralytics YOLOv8.2.74 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=/content/drive/MyDrive/datasets/person_Detection/yolov8n (1).pt, data=/content/drive/MyDrive/datasets/person_Detection/data.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save

100%|██████████| 755k/755k [00:00<00:00, 15.7MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

100%|██████████| 6.25M/6.25M [00:00<00:00, 74.3MB/s]


AMP: checks passed ✅


...

                   all        155        503      0.923      0.867      0.943      0.734



10 epochs completed in 0.039 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.2.74 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:05<00:00,  1.03s/it]


                   all        155        503      0.923      0.867      0.943      0.734
Speed: 0.3ms preprocess, 3.3ms inference, 0.0ms loss, 6.8ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x78f767cd63b0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [10]:
model('/content/drive/MyDrive/datasets/whole_image.jpg')  # predict on an image


image 1/1 /content/drive/MyDrive/datasets/whole_image.jpg: 640x640 3 persons, 8.0ms
Speed: 2.2ms preprocess, 8.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'person'}
 obb: None
 orig_img: array([[[ 56, 150, 149],
         [ 59, 153, 152],
         [ 64, 158, 157],
         ...,
         [ 34,  84,  96],
         [ 35,  86, 102],
         [ 39,  90, 106]],
 
        [[ 63, 157, 156],
         [ 66, 160, 159],
         [ 70, 164, 163],
         ...,
         [ 22,  72,  84],
         [ 24,  73,  87],
         [ 25,  77,  90]],
 
        [[ 75, 172, 170],
         [ 77, 174, 172],
         [ 81, 178, 176],
         ...,
         [ 15,  67,  74],
         [ 18,  69,  79],
         [ 22,  73,  83]],
 
        ...,
 
        [[182, 214, 220],
         [179, 211, 217],
         [175, 206, 215],
         ...,
         [157, 196, 218],
         [156, 195, 217],
         [156, 195, 217]],
 
        [[182, 214, 220],
         [178, 210, 216],
         [173, 204, 213],
         ...,
         [151, 192

## Training yoloV8

In [23]:
import os
import shutil

# Path to the directory you want to delete
directory_path = '/content/drive/MyDrive/datasets/.ipynb_checkpoints'

# Use shutil.rmtree to remove the directory and its contents
shutil.rmtree(directory_path, ignore_errors=True)  # ignore_errors handles potential errors if the directory doesn't exist
print(f"Directory '{directory_path}' and its contents have been removed.")




Directory '/content/drive/MyDrive/datasets/.ipynb_checkpoints' and its contents have been removed.


In [29]:
import os
import shutil

def filter_out_classes(images_dir, labels_dir, exclude_class_ids, output_images_dir, output_labels_dir):
    if not os.path.exists(output_images_dir):
        os.makedirs(output_images_dir)
    if not os.path.exists(output_labels_dir):
        os.makedirs(output_labels_dir)

    label_files = [f for f in os.listdir(labels_dir) if f.endswith('.txt')]

    for label_file in label_files:
        label_path = os.path.join(labels_dir, label_file)
        image_file = label_file.replace('.txt', '.jpg')  # Adjust extension if needed
        image_path = os.path.join(images_dir, image_file)

        with open(label_path, 'r') as file:
            lines = file.readlines()

        # Filter out lines with class IDs in the exclude_class_ids list
        filtered_lines = [line for line in lines if int(line.split()[0]) not in exclude_class_ids]

        if filtered_lines:
            if os.path.exists(image_path):
                shutil.copy(image_path, output_images_dir)

            new_label_path = os.path.join(output_labels_dir, label_file)
            with open(new_label_path, 'w') as file:
                file.writelines(filtered_lines)

def prepare_dataset(source_images_dir, source_labels_dir, exclude_class_ids, train_images_dir, train_labels_dir, val_images_dir, val_labels_dir):
    for directory in [train_images_dir, train_labels_dir, val_images_dir, val_labels_dir]:
        if not os.path.exists(directory):
            os.makedirs(directory)

    all_label_files = [f for f in os.listdir(source_labels_dir) if f.endswith('.txt')]
    import random
    random.shuffle(all_label_files)
    split_index = int(len(all_label_files) * 0.8)
    train_label_files = all_label_files[:split_index]
    val_label_files = all_label_files[split_index:]

    for label_files, img_output_dir, lbl_output_dir in [(train_label_files, train_images_dir, train_labels_dir), (val_label_files, val_images_dir, val_labels_dir)]:
        for label_file in label_files:
            label_path = os.path.join(source_labels_dir, label_file)
            image_file = label_file.replace('.txt', '.jpg')  # Adjust extension if needed
            image_path = os.path.join(source_images_dir, image_file)

            with open(label_path, 'r') as file:
                lines = file.readlines()

            # Filter out lines with class IDs in the exclude_class_ids list
            filtered_lines = [line for line in lines if int(line.split()[0]) not in exclude_class_ids]

            if filtered_lines:
                if os.path.exists(image_path):
                    shutil.copy(image_path, img_output_dir)

                new_label_path = os.path.join(lbl_output_dir, label_file)
                with open(new_label_path, 'w') as file:
                    file.writelines(filtered_lines)

# Directories and parameters

exclude_class_ids = [0,3,7,8,9]  # ID of the class you dont want to detect

# Directories and parameters
source_images_dir = '/content/drive/MyDrive/datasets/images'
source_labels_dir = '/content/drive/MyDrive/datasets/labels'


train_images_dir = '/content/drive/MyDrive/datasets/PPE_Detection/train/images'
train_labels_dir = '/content/drive/MyDrive/datasets/PPE_Detection/train/labels'
val_images_dir = '/content/drive/MyDrive/datasets/PPE_Detection/val/images'
val_labels_dir = '/content/drive/MyDrive/datasets/PPE_Detection/val/labels'

# Prepare the dataset
prepare_dataset(source_images_dir, source_labels_dir, exclude_class_ids, train_images_dir, train_labels_dir, val_images_dir, val_labels_dir)


In [31]:
import os

def update_annotation_classes(input_file, output_file, class_map):
    with open(input_file, 'r') as file:
        lines = file.readlines()

    with open(output_file, 'w') as file:
        for line in lines:
            parts = line.split()
            if parts:  # Check if the line is not empty
                try:
                    # Update only the first column (class index)
                    class_index = int(parts[0])
                    updated_class_index = str(class_map.get(class_index, class_index))
                    updated_parts = [updated_class_index] + parts[1:]
                except ValueError:
                    # In case of unexpected values in the first column
                    updated_parts = parts
            else:
                updated_parts = parts

            updated_line = ' '.join(updated_parts)
            file.write(updated_line + '\n')

def process_directory(directory_path, class_map):
    for filename in os.listdir(directory_path):
        if filename.endswith('.txt'):  # Assuming your annotations are in .txt files
            input_file = os.path.join(directory_path, filename)
            output_file = os.path.join(directory_path, filename)
            update_annotation_classes(input_file, output_file, class_map)
            print(f'Processed {input_file}')

# Example usage:
class_map = {1: 0, 2: 1, 4: 2, 5: 3,6:4}   # Map old class indices to new indices
directory_path = '/content/drive/MyDrive/datasets/PPE_Detection/val/labels'  # Update this to your directory
process_directory(directory_path, class_map)




Processed /content/drive/MyDrive/datasets/PPE_Detection/val/labels/005126.txt
Processed /content/drive/MyDrive/datasets/PPE_Detection/val/labels/image_108_jpg.rf.a2a7f9dd16e7a136d02fb42a810d1644.txt
Processed /content/drive/MyDrive/datasets/PPE_Detection/val/labels/ppe_0941_jpg.rf.c7fcf7d8f89d423c80586709b693b697.txt
Processed /content/drive/MyDrive/datasets/PPE_Detection/val/labels/001773.txt
Processed /content/drive/MyDrive/datasets/PPE_Detection/val/labels/005268.txt
Processed /content/drive/MyDrive/datasets/PPE_Detection/val/labels/ppe_1130_jpg.rf.88358e5b86cdc4d55d4689abe5cb87e4.txt
Processed /content/drive/MyDrive/datasets/PPE_Detection/val/labels/001190.txt
Processed /content/drive/MyDrive/datasets/PPE_Detection/val/labels/001426.txt
Processed /content/drive/MyDrive/datasets/PPE_Detection/val/labels/001438.txt
Processed /content/drive/MyDrive/datasets/PPE_Detection/val/labels/001199.txt
Processed /content/drive/MyDrive/datasets/PPE_Detection/val/labels/001889.txt
Processed /cont

In [34]:
# Load YOLOv8n-cls, train it on mnist160 for 10 epochs and predict an image with it
from ultralytics import YOLO
model = YOLO("/content/drive/MyDrive/datasets/PPE_Detection/Copy of yolov8n (1).pt")  # load a pretrained YOLOv8n classification model
model.train(data='/content/drive/MyDrive/datasets/PPE_Detection/data.yaml', epochs=50)



Ultralytics YOLOv8.2.74 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=/content/drive/MyDrive/datasets/PPE_Detection/Copy of yolov8n (1).pt, data=/content/drive/MyDrive/datasets/PPE_Detection/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, sa

train: Scanning /content/drive/MyDrive/datasets/person_Detection/train/labels.cache... 397 images, 1 backgrounds, 0 corrupt: 100%|██████████| 397/397 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/drive/MyDrive/datasets/person_Detection/val/labels.cache... 155 images, 0 backgrounds, 0 corrupt: 100%|██████████| 155/155 [00:00<?, ?it/s]


Plotting labels to runs/detect/train/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001111, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/train
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        155        503      0.988      0.958      0.989        0.9



50 epochs completed in 0.204 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.3MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.2.74 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3,006,623 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:07<00:00,  1.46s/it]


                   all        155        503      0.988      0.958      0.989        0.9
              hard-hat        155        503      0.988      0.958      0.989        0.9
Speed: 0.3ms preprocess, 3.6ms inference, 0.0ms loss, 9.4ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x78f698d9e320>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [35]:
model('/content/drive/MyDrive/datasets/whole_image.jpg')  # predict on an image


image 1/1 /content/drive/MyDrive/datasets/whole_image.jpg: 640x640 3 hard-hats, 40.2ms
Speed: 2.4ms preprocess, 40.2ms inference, 2.3ms postprocess per image at shape (1, 3, 640, 640)


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'hard-hat', 1: 'gloves', 2: 'glasses', 3: 'boots', 4: 'vest'}
 obb: None
 orig_img: array([[[ 56, 150, 149],
         [ 59, 153, 152],
         [ 64, 158, 157],
         ...,
         [ 34,  84,  96],
         [ 35,  86, 102],
         [ 39,  90, 106]],
 
        [[ 63, 157, 156],
         [ 66, 160, 159],
         [ 70, 164, 163],
         ...,
         [ 22,  72,  84],
         [ 24,  73,  87],
         [ 25,  77,  90]],
 
        [[ 75, 172, 170],
         [ 77, 174, 172],
         [ 81, 178, 176],
         ...,
         [ 15,  67,  74],
         [ 18,  69,  79],
         [ 22,  73,  83]],
 
        ...,
 
        [[182, 214, 220],
         [179, 211, 217],
         [175, 206, 215],
         ...,
         [157, 196, 218],
         [156, 195, 217],
         [156, 195, 217]],
 
        [[182, 214, 220],
         [178, 210, 216],
      

In [40]:
import os
import shutil

# Path to the directory you want to delete
directory_path = '/content/runs'

# Use shutil.rmtree to remove the directory and its contents
shutil.rmtree(directory_path, ignore_errors=True)  # ignore_errors handles potential errors if the directory doesn't exist
print(f"Directory '{directory_path}' and its contents have been removed.")




Directory '/content/runs' and its contents have been removed.


In [41]:
import os
import shutil
from sklearn.model_selection import train_test_split

def split_data(image_dir, annotation_dir, output_dir, test_size=0.2):
    # Create output directories
    train_image_dir = os.path.join(output_dir, 'train/images')
    val_image_dir = os.path.join(output_dir, 'val/images')
    train_annotation_dir = os.path.join(output_dir, 'train/labels')
    val_annotation_dir = os.path.join(output_dir, 'val/labelss')

    os.makedirs(train_image_dir, exist_ok=True)
    os.makedirs(val_image_dir, exist_ok=True)
    os.makedirs(train_annotation_dir, exist_ok=True)
    os.makedirs(val_annotation_dir, exist_ok=True)

    # List all image files
    images = [f for f in os.listdir(image_dir) if f.endswith('.jpg') or f.endswith('.png')]

    # Split data
    train_images, val_images = train_test_split(images, test_size=test_size, random_state=42)

    # Copy training images and annotations
    for image in train_images:
        shutil.copy(os.path.join(image_dir, image), os.path.join(train_image_dir, image))
        annotation = image.replace('.jpg', '.txt').replace('.png', '.txt')
        shutil.copy(os.path.join(annotation_dir, annotation), os.path.join(train_annotation_dir, annotation))

    # Copy validation images and annotations
    for image in val_images:
        shutil.copy(os.path.join(image_dir, image), os.path.join(val_image_dir, image))
        annotation = image.replace('.jpg', '.txt').replace('.png', '.txt')
        shutil.copy(os.path.join(annotation_dir, annotation), os.path.join(val_annotation_dir, annotation))

    print(f"Data split into {len(train_images)} training and {len(val_images)} validation samples.")

# Example usage
image_directory = '/content/drive/MyDrive/datasets/images'
annotation_directory = '/content/drive/MyDrive/datasets/labels'
output_directory = '/content/output'
split_data(image_directory, annotation_directory, output_directory, test_size=0.2)


Data split into 332 training and 84 validation samples.


In [42]:
# Load YOLOv8n-cls, train it on mnist160 for 10 epochs and predict an image with it
from ultralytics import YOLO
model = YOLO("/content/yolov8n.pt")  # load a pretrained YOLOv8n classification model
model.train(data='/content/data.yaml', epochs=5)



Ultralytics YOLOv8.2.74 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=/content/yolov8n.pt, data=/content/data.yaml, epochs=5, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_

train: Scanning /content/output/train/labels... 332 images, 5 backgrounds, 0 corrupt: 100%|██████████| 332/332 [00:00<00:00, 1762.84it/s]


train: New cache created: /content/output/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/output/val/labels... 84 images, 2 backgrounds, 0 corrupt: 100%|██████████| 84/84 [00:00<00:00, 659.10it/s]

val: New cache created: /content/output/val/labels.cache


...

                   all         84        754       0.63      0.384      0.421      0.273



5 epochs completed in 0.021 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.2.74 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3,007,598 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:03<00:00,  1.19s/it]


                   all         84        754      0.634      0.385      0.421      0.273
                person         82        246      0.762      0.882      0.903      0.707
              hard-hat         70        165      0.824      0.776      0.857      0.639
                gloves         33         64      0.129     0.0938     0.0844     0.0341
                  mask          2          2          1          0          0          0
                 boots         45        192      0.492      0.469      0.417      0.192
                  vest         19         27      0.737      0.333      0.444      0.223
              ppe-suit         28         58      0.497      0.138       0.24      0.115
Speed: 0.2ms preprocess, 2.2ms inference, 0.0ms loss, 9.1ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 5, 6, 7])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x78f67e74d030>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
  